# Processamento de Dados Pós-Imputação

Autores: Rafaella Ormond e Jose Jaime Martinez-Magana <br>
***Descrição:***<br>
Este script processa dados imputados do servidor TOPMED, convertendo arquivos VCF para o formato PLINK e realizando filtragem de controle de qualidade.

**ENTRADA:** Arquivos de saída da imputação do servidor TOPMed (por exemplo, VCFs específicos por cromossomo ou arquivos de dosagem).<br>
**SAÍDA:** Arquivos PLINK com controle de qualidade, filtrados e prontos para análises subsequentes.


### ***Requisitos:***
### Baixar o Plink
Você pode baixar o Plink versão 1.9 e 2.0 seguindo as etapas do site oficial.<br>
Para instalar o plink2 [acesse aqui](https://www.cog-genomics.org/plink/2.0/)<br>
Para instalar o plink1.9 [acesse aqui](https://www.cog-genomics.org/plink/1.9/) 

### Baixar o BCFtools
**BCFtools** é outra ferramenta popular para manipulação de arquivos VCF/BCF:  
- [Download do BCFtools](http://samtools.github.io/bcftools/)  

### Resultados da Imputação TOPMED
Este script assume que você já concluiu a imputação usando o servidor TOPMED [link aqui](https://imputation.biodatacatalyst.nhlbi.nih.gov/#!) e fez o download dos resultados<br>
Após a conclusão da imputação, será necessário baixar:

- O **Relatório de Controle de Qualidade** (por exemplo, `qcreport.html`)
- As **estatísticas resumidas de QC** (por exemplo, `chunks-excluded.txt`, `snps-excluded.txt`, `typed-only.txt`)
- Os **arquivos VCF imputados**, um por cromossomo (por exemplo, `chr1.dose.vcf.gz`, `chr2.dose.vcf.gz`, ...)

Esses arquivos estão disponíveis por meio de links diretos na interface do servidor após a conclusão do trabalho. Certifique-se de salvá-los em uma estrutura de pastas consistente para análises subsequentes.

## Opções de Software para QC

Você pode realizar o controle de qualidade pós-imputação usando uma das seguintes opções:

1) Plink
2) BCFtools

| Ferramenta | Prós ✅ | Contras ❌ |
|-----------|--------|---------|
| **PLINK** | - Amplamente utilizado em pipelines de GWAS.<br>- Muito rápido para grandes conjuntos de dados.<br>- Diversas funções integradas de QC genético (MAF, HWE, missingness, etc.).<br>- Pode gerar múltiplos formatos de saída. | - Requer conversão de VCF para formato PLINK primeiro.<br>- Manipula menos bem estruturas complexas de VCF em comparação com o BCFtools. |
| **BCFtools** | - Trabalha diretamente com arquivos VCF/BCF compactados.<br>- Muito eficiente para filtragem e subamostragem de grandes arquivos de variantes.<br>- Flexível para automação em scripts.<br>- Suporta integração via *streaming* com outras ferramentas. | - Exige mais scripts manuais para etapas específicas de QC em GWAS.<br>- Menos automatizado para fluxos de trabalho de associação genética em comparação com o PLINK. |

---

💡 **Dica:**  
- Se você pretende avançar rapidamente para análises de GWAS, **PLINK** é geralmente a escolha mais fácil.  
- Se você busca máxima flexibilidade e trabalha frequentemente com arquivos VCF brutos, **BCFtools** é uma opção poderosa.

Etapas da Análise:¶
1. **Descompactar os arquivos de imputação**<br>
2. **Plink**<br>
   2.1 Configuração do usuário<br>
   2.2 Criar arquivos PLINK e juntar todos os cromossomos<br>
   2.3 Juntar arquivos PLINK e padronizar os IDs das variantes<br>
   2.4 Realizar *LD pruning* no conjunto de dados PLINK mesclado
   
3. **BCFTools**<br>
   3.1 Configuração do usuário<br>
   3.2 Controle de Qualidade<br>
   3.3 Converter BCF filtrado para arquivos PLINK separados por cromossomo
   3.4 Juntar arquivos PLINK separados por cromossomo<br>
   3.5 Padronizar IDs das variantes e manter apenas SNPs


### 1. Descompactar os arquivos de imputação
***Descrição:***<br>
Descompacte os arquivos de saída gerados pela imputação.

> **Nota:** Alguns servidores de imputação, como o TOPMed, podem criptografar os arquivos de saída com uma senha por motivos de segurança.  
> A senha geralmente é fornecida na página de resultados do seu trabalho ou no e-mail de confirmação.  
> Certifique-se de substituir a senha de exemplo acima pela senha específica do seu conjunto de dados.

Ajuste o caminho e os nomes dos arquivos conforme necessário.

In [ ]:
cd ${input_path}

# descompactar todos os arquivos
# adicione a senha do seu trabalho na linha abaixo
password='XXX'
for chr in *.zip
do
    echo unzip -P ${password} -o ${chr}
    unzip -P ${password} -o ${chr}
done

### 2. PLINK

### 2.1 Configuração do Usuário

***Descrição:***  
Defina os parâmetros e ajuste os caminhos e arquivos de acordo com a sua análise.

In [ ]:
# definir o caminho de entrada
input_path="/path_to_your_data/imputation_results"

# definir o diretório para arquivos divididos por cromossomo
splitted_path="/path_to_your_data/splitted_files"

# definir o caminho para o executável do plink2
plink2="/path_to_your_data/plink2"

# definir o diretório para os arquivos com os cromossomos mesclados
merged_path="/path_to_your_data/merged_files"

# definir o diretório para os arquivos após o LD pruning
ldpruned_path="/path_to_your_data/merged_files_ldpruned"


### 2.2 Criar arquivos PLINK e juntar todos os cromossomos
***Descrição:***<br>
Converta os arquivos `.dose.vcf.gz` para o formato PLINK e junte todos os cromossomos em um único arquivo.

Ajuste o caminho e os arquivos conforme necessário.


In [ ]:
# transformar arquivos de dosagem em formato PLINK
for chr in {1..22}
do
    ${plink2} --vcf ${input_path}/chr${chr}.dose.vcf.gz \
              --make-pgen \
              --id-delim '_' \
              --rm-dup 'force-first' \
              --snps-only \
              --out ${splitted_path}/chr${chr}.dose.plink
done


### 2.3 Juntar arquivos PLINK e padronizar IDs das variantes

***Descrição:*** <br>
Este script acessa o diretório que contém os arquivos PLINK divididos por cromossomo, cria uma lista desses arquivos, junta todos em um único conjunto de arquivos e padroniza os IDs das variantes no formato `chr:bp:ref:alt`, garantindo consistência nas análises subsequentes.

In [ ]:
# mover para o diretório com arquivos divididos por cromossomo
cd ${splitted_path}

# criar lista de arquivos PLINK, se necessário
# for chr in {2..22}; do echo chr${chr}.dose.plink; done > plink_files.list

# carregar caminho do executável plink2
plink2="/path_to_your_data/plink2"

# transformar arquivos de dosagem em formato PLINK e juntar todos os cromossomos
${plink2} --pfile chr1.dose.plink \
          --pmerge-list plink_files.list pfile \
          --multiallelics-already-joined \
          --out ${merged_path}/merged.chr.dose.plink

# alterar todos os IDs das variantes para o formato chr:bp:ref:alt
${plink2} --pfile ${merged_path}/merged.chr.dose.plink \
          --set-all-var-ids 'chr@:#:$1:$2' \
          --snps-only 'just-acgt' \
          --make-pgen \
          --out ${merged_path}/merged.chr.dose.plink.norsids.snpsonly

### 3. BCFTools

### 3.1 Configuração do Usuário

***Descrição:***<br>
Defina os parâmetros e ajuste os caminhos e arquivos de acordo com a sua análise.


In [ ]:
input='/path_to_your_data/00impfiles'
filtered_path='/path_to_your_data/02impfilesfiltered'
plink_path="/path_to_your_data/plink2"
plink_out="/path_to_your_data/plink_per_chr"
merged_path="/path_to_your_data/merged_plink"

### 3.2 Controle de Qualidade
**Descrição:**

Este script processa sequencialmente os arquivos de variantes imputadas específicos por cromossomo, aplicando filtros de qualidade utilizando o `bcftools`. <br>
As variantes são filtradas com base na qualidade da imputação (R²) e na frequência do alelo menor (MAF).<br> 
Os arquivos filtrados são então compactados e indexados, preparando-os para análises genéticas subsequentes.

In [ ]:
# filter BRHC imputed variants
# set the paths for input and output data

# loop over each chromosome sequentially
for chr in {1..22}; do
    bcftools view -i 'R2>0.8 & MAF>0.001' --threads 30 -Oz -o ${filtered_path}/cohort.chr${chr}.dose.filtered.vcf.gz ${input}/chr${chr}.dose.vcf.gz
    bcftools +fill-tags --threads 30 ${filtered_path}/cohort.chr${chr}.dose.filtered.vcf.gz --write-index -Ob -o ${filtered_path}/cohort.chr${chr}.dose.filtered.tag.bcf.gz -- -t AC,AN
done

### 3.3 Converter BCF filtrado para arquivos PLINK separados por cromossomo
***Descrição:***<br>

Converte os arquivos BCF filtrados para o formato `.pgen` utilizando o PLINK.<br>
Altere **"cohort"** para o nome do arquivo correspondente.

In [ ]:
# filtrar variantes imputadas do BRHC
# definir os caminhos para os dados de entrada e saída

# loop sobre cada cromossomo, sequencialmente
for chr in {1..22}; do
    echo "Processando o cromossomo ${chr}..."

    # filtra variantes pela qualidade da imputação (R2) e pela frequência do alelo menor (MAF)
    bcftools view \
        -i 'R2>0.8 & MAF>0.001' \
        --threads 30 \
        -Oz \
        -o ${filtered_path}/cohort.chr${chr}.dose.filtered.vcf.gz \
        ${input}/chr${chr}.dose.vcf.gz

    # adiciona as tags de contagem alélica (AC) e número total de alelos (AN)
    bcftools +fill-tags \
        --threads 30 \
        ${filtered_path}/cohort.chr${chr}.dose.filtered.vcf.gz \
        --write-index \
        -Ob \
        -o ${filtered_path}/cohort.chr${chr}.dose.filtered.tag.bcf.gz \
        -- -t AC,AN

    echo "Cromossomo ${chr} concluído."
done

### 3.4 Juntar arquivos PLINK separados por cromossomo
***Descrição:***  
Junta os arquivos `.pgen` individuais de cada cromossomo em um único conjunto de arquivos PLINK.

Modifique os nomes de acordo com os seus arquivos.

In [ ]:
# mover para o diretório
mkdir -p ${merged_path}

# Criar lista de arquivos para juntar cromossomos (excluindo o cromossomo 1, que será o arquivo base)
for chr in {2..22}; do
    echo "${plink_out}/cohort_chr${chr}.plink" >> ${merged_path}/cohort_merge_list.txt
done

# Removendo todos os duplicados
for chr in {1..22}
do
plink2 --pfile ${datapath}/cohort_chr${chr}.plink --rm-dup 'exclude-all' --make-pgen --out cohort_nodup_chr${chr} \
done

# Gerando o arquivo de lista para a mesclagem
current_path= seu_caminho
for chr in {2..22}
do
echo "${current_path}/cohort_nodup_chr${chr}" >> cohort_merging-09182025.txt
done

# Mesclando os arquivos
plink2 --pfile cohort_nodup_chr1 --pmerge-list cohort_merging.txt 'pfile' --make-pgen --out cohort_nodup_merged


### 3.5 Padronizar IDs das variantes e manter apenas SNPs
***Descrição:*** <br>
Atualiza os IDs das variantes para o formato `@:#:$r:$a`, filtra para incluir apenas SNPs dos tipos A/C/G/T e recria o arquivo `.pgen`.

In [ ]:
${plink_path} \
    --pfile ${merged_path}/cohort_merged_allchr \
    --set-all-var-ids \
    --snps-only just-acgt \
    --make-pgen \
    --out ${merged_path}/cohort_merged_allchr.snpsonly